In [1]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'



In [2]:
#model = tf.keras.models.load_model('E:\Python\Modelos\Test.Lenet.2.h5')
#model = tf.keras.models.load_model('E:\Python\Modelos\Test.Alexnet.h5')
#model = tf.keras.models.load_model('E:\Python\Modelos\Test1.2.h5')
#model = tf.keras.models.load_model('E:\Python\Modelos\Test2.2.h5')
model = tf.keras.models.load_model('E:\Python\Modelos\Test2.5.h5')
#model = tf.keras.models.load_model('E:\Python\Modelos\Test3.1.h5')
#model = tf.keras.models.load_model('E:\Python\Modelos\Test3.2.h5')
#model = tf.keras.models.load_model('E:\Python\Modelos\Test6.1.h5')

In [3]:
CLASS_LABELS = ['Abuse','Arrest','Arson','Assault',
                'Burglary','Explosion','Fighting',"Normal",
                'RoadAccidents','Robbery','Shooting',
                'Shoplifting','Stealing','Vandalism']
CLASS_LABELS2 = ['Anomalia','Normal']

#PATH_IMAGE = "/Users/harry.lopez/Documents/Dataset/Test Video/Asalto.mp4"
#PATH_IMAGE = "/Users/harry.lopez/Documents/Dataset/Test Video/ExplosionesVideo.mp4"
PATH_IMAGE = "E:\Python\Videos\RoadAccidents022_x264.mp4"
PREDICTION_IMAGE = []
PREDICTION_TEXT = []
PREDICTION_VIDEO = []

In [4]:
import matplotlib.pyplot as plt
def display_images(img_list,title_list, cmap='gray', cols = 5, fig_size = (20, 20) ):
    """
    Display images in img_list
    """
    i = 1  # for subplot

    num_images = len(img_list)
    #num_rows = int(num_images / cols)
    num_rows = 8

    plt.figure(figsize=fig_size)
    plt.subplots_adjust(top = 1.0)

    for image in img_list:
        if i < 40:
            image_file_name = title_list[i - 1]
            plt.subplot(num_rows, cols, i)        
            plt.title(image_file_name,fontsize=15,color="green", pad='2.0')
            plt.imshow(image, cmap=cmap)        
        i += 1     

    plt.show()

In [5]:
def validateModel(pathImage, modelFunc, nameModel, classes):
    #cap = cv2.VideoCapture('/Users/harry.lopez/Documents/Dataset/Dataset/Vandalism/Vandalism033_x264.mp4')
    cap = cv2.VideoCapture(pathImage)
    ret, frame = cap.read()
    count = 0
    predictionIds = []
    countFrames = 0;
    frameArray = []
    predictionFrame = []

    # Mientras haya fotogramas
    while ret:
        # Redimensiona el fotograma a las dimensiones de entrada del modelo
        frame = cv2.resize(frame, (64, 64))

        # Convierte el fotograma a un tensor de una sola dimensión
        input_data = np.expand_dims(frame, axis=0)

        # Hace una predicción sobre el fotograma
        prediction = modelFunc.predict(input_data, verbose = 0)

        #print('Prediction:', prediction)
        index = indices = np.where(prediction[0] == prediction[0].max())
        predictionIds.append(index)
        #print('index', CLASS_LABELS[index[0][0]])

        if countFrames == 50:
            predictionFrame.append(classes[index[0][0]] + '=Frame:' + str(count))
            frameArray.append(frame)
            countFrames = 0
        else:
            countFrames += 1
        ret, frame = cap.read()
        count += 1


    # Libera el video
    print("Cantidad de Fotogramas:", count)
    print("Modelo de Prueba:", nameModel)
    cap.release()
    display_images(frameArray,predictionFrame)

In [6]:
def validateModelVideo(pathImage, modelFunc, nameModel, 
                       classes, width, height):
    cap = cv2.VideoCapture(pathImage)
    
    count = 0
    predictionIds = []
    countFrames = 0;
    frameArray = []
    predictionFrame = []

    # Mientras haya fotogramas
    while (cap.isOpened()):
        
        ret, frame = cap.read()
        # Redimensiona el fotograma a las dimensiones de entrada del modelo
        frame2 = cv2.resize(frame, (width, height))
        
        frame = cv2.resize(frame, (64, 64))

        # Convierte el fotograma a un tensor de una sola dimensión
        input_data = np.expand_dims(frame, axis=0)
        
        # Hace una predicción sobre el fotograma
        prediction = modelFunc.predict(input_data, verbose = 0)
        
        index = indices = np.where(prediction[0] == prediction[0].max())
        predictionIds.append(index)

        #Obtener el index de os tres mayores puntajes
        scores = np.argsort(prediction[0])[::-1][:3]
        
        #obtener los  valores de los 3 mayores puntajes
        predictionValues = prediction[0][scores]
        
        #Obtener en texto los  valores de las tre mayores predicciones
        predictionString = classes[scores[0]] + ' Score:' 
        + "{:.2f}".format(predictionValues[0] * 100)
        predictionString1 = classes[scores[1]] + ' Score:' 
        + "{:.2f}".format(predictionValues[1] * 100)
        predictionString2 = classes[scores[2]] + ' Score:' 
        + "{:.2f}".format(predictionValues[2] * 100)
        
        #POner en el frame los valores obtenidos
        cv2.putText(frame2, predictionString, (45,45), 
                    cv2.FONT_HERSHEY_TRIPLEX, 2.0, (0,0,255), 1)
        cv2.putText(frame2, predictionString1, (45,100), 
                    cv2.FONT_HERSHEY_TRIPLEX, 2.0, (0,0,255), 1)
        cv2.putText(frame2, predictionString2, (45,155), 
                    cv2.FONT_HERSHEY_TRIPLEX, 2.0, (0,0,255), 1)

        #Almacenar los index de las predicciones
        PREDICTION_VIDEO.append(scores)
        
        #Contar los frames evalulados
        count += 1
        
        #Mostrar el frame analizado
        cv2.imshow('Capture', frame2)
        
        #Opciòn para salir en el proceso 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


    # Libera el video
    print("Cantidad de Fotogramas:", count)
    print("Modelo de Prueba:", nameModel)
    cap.release()
    display_images(frameArray,predictionFrame)

In [ ]:
validateModelVideo(PATH_IMAGE, model, 
                   'Model Test 3-2-3', 
                   CLASS_LABELS, 1024, 780)